In [1]:
import pandas as pd
BRCA_FR_SUPERFILTRATO_WG=pd.read_csv('/content/BRCA_FR_AF05_CLINSIG_NA_FILTRO_sample_inputmatrix.txt',sep='\t')

In [2]:
BRCA_FR_SUPERFILTRATO_WG

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,202468579,202468579,C,A,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
420794,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
420795,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
420796,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
420797,BRCA-FR,SA595120,SA595120,GRCh37,SNP,6,42376519,42376519,C,T,SOMATIC


In [3]:
ESONI=pd.read_csv('/content/drive/MyDrive/ESONI.tsv',sep='\t')

In [4]:
BRCA_FR_SUPERFILTRATO_WG['chrom']='chr'+ BRCA_FR_SUPERFILTRATO_WG['chrom'].astype(str)

In [5]:
ESONI['chrom']=ESONI['chrom'].astype(str)

In [6]:
ESONI['exonStarts']=ESONI['exonStarts'].astype(int)

In [7]:
ESONI['exonEnds']=ESONI['exonEnds'].astype(int)

In [8]:
BRCA_FR_SUPERFILTRATO_WG['pos_start']=BRCA_FR_SUPERFILTRATO_WG['pos_start'].astype(int)
BRCA_FR_SUPERFILTRATO_WG['pos_end']=BRCA_FR_SUPERFILTRATO_WG['pos_end'].astype(int)

In [9]:
import pandas as pd
import numpy as np
from multiprocessing import Pool

# Definizione della funzione per il confronto dei blocchi
def compare_blocks(block_BRCA, block_ESONI):
    # Resto del codice rimane invariato
    merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
    filtered_block = merged_block[
        (merged_block['pos_start'] >= merged_block['exonStarts']) &
        (merged_block['pos_end'] <= merged_block['exonEnds'])
    ]
    return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start',
       'pos_end', 'ref', 'alt', 'Type']]

# Dimensione dei blocchi
block_size = 1000

# Calcolo del numero di blocchi per ciascun DataFrame
num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

# Creazione della lista di indici per i blocchi
block_indices = []
for i in range(num_blocks_BRCA):
    start_idx_BRCA = i * block_size
    end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

    for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

# Funzione wrapper per la chiamata a compare_blocks con gli indici
def compare_blocks_wrapper(indices):
    block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
    block_ESONI = ESONI.iloc[indices[2]:indices[3]]

    return compare_blocks(block_BRCA, block_ESONI)

# Utilizzo di multiprocessing per eseguire operazioni parallele
with Pool() as pool:
    results = pool.map(compare_blocks_wrapper, block_indices)

# Combinazione dei risultati in un unico DataFrame
final_result = pd.concat(results, ignore_index=True)

In [10]:
final_result

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,145688679,145688679,A,C,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,33611041,33611041,A,C,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,36898132,36898132,T,G,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,36898132,36898132,T,G,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,118409517,118409517,A,C,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
80628,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr19,8127477,8127477,C,T,SOMATIC
80629,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr19,8127477,8127477,C,T,SOMATIC
80630,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr19,8127477,8127477,C,T,SOMATIC
80631,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr19,8127477,8127477,C,T,SOMATIC


In [11]:
# Salva il DataFrame come file di testo separato da virgole (CSV)
final_result.to_csv('brca_fr_we_na_af05_clinsig.txt', sep='\t', index=False)  # Separa le colonne con il carattere di tabulazione ('\t')

In [24]:
import pandas as pd
BRCA_FR_SUPERFILTRATO_WE=pd.read_csv('/content/brca_fr_we_superfiltrato.txt',sep='\t')

In [25]:
BRCA_FR_SUPERFILTRATO_WE

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,145688679,145688679,A,C,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,33611041,33611041,A,C,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,36898132,36898132,T,G,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,36898132,36898132,T,G,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr1,118409517,118409517,A,C,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
80400,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr17,37881000,37881000,G,T,SOMATIC
80401,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr19,8127477,8127477,C,T,SOMATIC
80402,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr19,8127477,8127477,C,T,SOMATIC
80403,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr19,8127477,8127477,C,T,SOMATIC


In [26]:
for i in range(BRCA_FR_SUPERFILTRATO_WE.shape[0]):
  BRCA_FR_SUPERFILTRATO_WE.loc[i,'chrom']= BRCA_FR_SUPERFILTRATO_WE.loc[i,'chrom'][3:]

In [27]:
BRCA_FR_SUPERFILTRATO_WE

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,145688679,145688679,A,C,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,33611041,33611041,A,C,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,36898132,36898132,T,G,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,36898132,36898132,T,G,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,118409517,118409517,A,C,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
80400,BRCA-FR,SA595120,SA595120,GRCh37,SNP,17,37881000,37881000,G,T,SOMATIC
80401,BRCA-FR,SA595120,SA595120,GRCh37,SNP,19,8127477,8127477,C,T,SOMATIC
80402,BRCA-FR,SA595120,SA595120,GRCh37,SNP,19,8127477,8127477,C,T,SOMATIC
80403,BRCA-FR,SA595120,SA595120,GRCh37,SNP,19,8127477,8127477,C,T,SOMATIC


In [28]:
BRCA_FR_SUPERFILTRATO_WE.to_csv('brca_fr_we_superfiltrato_new.txt', sep='\t', index=False)

In [29]:
import pandas as pd
BRCA_FR_05_NA_CLINSIG_WE=pd.read_csv('/content/brca_fr_we_na_af05_clinsig.txt',sep='\t')

In [30]:
for i in range(BRCA_FR_05_NA_CLINSIG_WE.shape[0]):
  BRCA_FR_05_NA_CLINSIG_WE.loc[i,'chrom']=BRCA_FR_05_NA_CLINSIG_WE.loc[i,'chrom'][3:]

In [31]:
BRCA_FR_05_NA_CLINSIG_WE.to_csv('brca_fr_we_NA_AF05_clinsig_new.txt', sep='\t', index=False)

In [32]:
BRCA_FR_05_NA_CLINSIG_WE

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,145688679,145688679,A,C,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,33611041,33611041,A,C,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,36898132,36898132,T,G,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,36898132,36898132,T,G,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,118409517,118409517,A,C,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
80628,BRCA-FR,SA595120,SA595120,GRCh37,SNP,19,8127477,8127477,C,T,SOMATIC
80629,BRCA-FR,SA595120,SA595120,GRCh37,SNP,19,8127477,8127477,C,T,SOMATIC
80630,BRCA-FR,SA595120,SA595120,GRCh37,SNP,19,8127477,8127477,C,T,SOMATIC
80631,BRCA-FR,SA595120,SA595120,GRCh37,SNP,19,8127477,8127477,C,T,SOMATIC
